<a href="https://colab.research.google.com/github/ChelseaOwiti/Colposcopy-Scan/blob/main/Copy_of_ISP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import tensorflow
import keras
import os
import glob
from skimage import io
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_path = './content/drive/MyDrive/train_new/train'

In [9]:
from keras.preprocessing.image  import ImageDataGenerator

train_datagen = ImageDataGenerator()
train_data = train_datagen.flow_from_directory('/content/drive/MyDrive/train_new/', classes = ['train'])

Found 70 images belonging to 1 classes.


In [15]:
train = ['Patient_One', 'Patient_Two', 'Patient_Three', 'Patient_Four']
patient_path = os.path.join(train_path, train[1], '*')
patient_path = glob.glob(patient_path)

In [16]:
image = io.imread(patient_path[4])

i, (im1) = plt.subplots(1)
i.set_figwidth(15)
im1.imshow(image)

IndexError: ignored

In [ ]:
#plotting images and rgb channels
i,(im1, im2, im3, im4) = plt.subplots(1,4, sharey =True)
i.set_figwidth(20)

im1.imshow(image) #original image
im2.imshow(image[;, ;, 0]) #red
im3.imshow(image[;, ;, 1]) #green
im4.imshow(image[;, ;, 2]) #blue
i.suptitle('Original and RGB channels')

In [ ]:
#Normalization
norm_image = ()

